In [1]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']

In [2]:
wordsRDD = sc.parallelize(wordsList, 4)

In [3]:
print type(wordsRDD)

<class 'pyspark.rdd.RDD'>


In [4]:
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  No attempt is made to follow proper
        pluralization rules.

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

print makePlural('cat')

cats


In [5]:
# One way of completing the function
def makePlural(word):
    return word + 's'

print makePlural('cat')

cats


In [6]:
from test_helper import Test

ImportError: No module named test_helper

In [7]:
pluralRDD = wordsRDD.map(makePlural)

In [8]:
print pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


In [9]:
pluralLambdaRDD = wordsRDD.map(lambda word: word + 's')

In [10]:
print pluralLambdaRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


In [11]:
pluralLengths = (pluralRDD
                 .map(lambda word: len(word))
                 .collect())
print pluralLengths

[4, 9, 4, 4, 4]


In [12]:
wordPairs = wordsRDD.map(lambda word: (word,1) )
print wordPairs.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


In [13]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0}: {1}'.format(key, list(value))

rat: [1, 1]
elephant: [1]
cat: [1, 1]


In [14]:
wordCountsGrouped = wordsGrouped.map(lambda (word ,c): (word, sum(c)))
print wordCountsGrouped.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [15]:
wordCounts = wordPairs.reduceByKey(lambda a,b: a+b)
print wordCounts.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [16]:
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .groupByKey()
                       .map(lambda (word, c):(word, sum(c)))
                       .reduceByKey(lambda a,b: a+b)
                       .collect())
print wordCountsCollected

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [17]:
uniqueWords = len(wordsRDD.distinct().collect())
print uniqueWords

3


In [18]:
from operator import add
totalCount = (wordCounts
              .map(lambda(w,c):c)
              .reduce(lambda x,y: x+y))
average = totalCount / float(uniqueWords)
print totalCount
print round(average, 2)

5
1.67


In [19]:
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (wordListRDD.map(lambda word: (word, 1))
                    .groupByKey()
                    .map(lambda (word,c): (word, sum(c)))
                    .reduceByKey(lambda x,y : x+y))
            
print wordCount(wordsRDD).collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [20]:
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^a-z0-9\s]','',text.lower().strip())
print removePunctuation('Hi, you!')
print removePunctuation(' No under_score!')

hi you
no underscore


In [26]:
import os.path
baseDir = os.path.join('/home')
print baseDir
inputPath = os.path.join('chris','distributed_system','shakespeare.txt')
fileName = os.path.join(baseDir, inputPath)
print fileName
shakespeareRDD = (sc
                  .textFile(fileName, 8)
                  .map(removePunctuation))
print '\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda (l, num): '{0}: {1}'.format(num, l))  # to 'lineNum: line'
                .take(15))

/home
/home/chris/distributed_system/shakespeare.txt
0: 1609
1: 
2: the sonnets
3: 
4: by william shakespeare
5: 
6: 
7: 
8: 1
9: from fairest creatures we desire increase
10: that thereby beautys rose might never die
11: but as the riper should by time decease
12: his tender heir might bear his memory
13: but thou contracted to thine own bright eyes
14: feedst thy lights flame with selfsubstantial fuel


In [27]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda line: line.split(' '))

In [28]:
shakespeareWordCount = shakespeareWordsRDD.count()

In [29]:
print shakespeareWordsRDD.top(5)

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']


In [30]:
print shakespeareWordCount

928908


In [31]:
shakeWordsRDD = shakespeareWordsRDD.filter(lambda word: len(word)>0)

In [32]:
shakeWordCount = shakeWordsRDD.count()

In [33]:
print shakeWordCount

882996


In [34]:
top15WordsAndCounts = wordCount(shakeWordsRDD).takeOrdered(15, lambda(w, s): -1 *s)

In [35]:
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts))

the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
